### Approximation AR(1) using Markov process
##### 2018/05/08 Tokuma Suzuki
UTGSE M1


### Outline
1.Introduction

2.Tauchen method

3.Rouwenhorst method

4.stationary distribution

### AR(1) process

Consider the following process.
\begin{equation*}
z_{t+1} = \rho z_t + \epsilon_{t+1}
\end{equation*}
- $\rho:$ persistence of AR(1) process
 Assume $\mid \rho \mid <1$.

- $\epsilon\sim N(0.\sigma^2)$ and i.i.d.

\begin{equation*}
\sigma^2_z =\frac{\sigma^2}{(1.0-ρ^2)}
\end{equation*}


### Approximate AR(1) using Markov process
Why do we approximate AR(1) process using Markov process?

- Hard to compute the model with continuous shock
- Markov process is closely related with dynamic programming.


### Tauchen method

1. Set grid number, nz of Markov process, $\rho$ and $\sigma$
2. Set upper and lower bound of realization, zmax, zmin 
 and create equally spaced grid, zgrid.
 
 2.1. Let zmin$ = -s\cdotσ_z$ and zmax $=-$zmin
3. Store width of between grid points, w.
4. Compute transition matrix, $\Pi$


### How to compute transition matrix
compute transition probability from state $i$ to $j$,
$p_{i,j}$ as follows.

$\Phi\left(\frac{z_1-\rho z_i + w/2}{\sigma}\right) 
\,\, \text{if $j =1$} $


$ \Phi\left(\frac{z_j-\rho z_i + w/2}{\sigma}\right) - \Phi\left(\frac{z_j-\rho z_i - w/2}{\sigma}\right)　\, \, \text{if $j =2 ,\ldots, nz-1$} $

$1-\Phi\left(\frac{z_{nz}-\rho z_i -w/2}{\sigma}\right) \,\,\text{if $j =nz$} $

In [1]:
using Distributions # for tauchen
using StatsBase # for accuracy check
using QuantEcon # check my code

In [2]:
function mytauchen(nz,ρ,σ,s::Int=3)  
    if nz==1
        throw(ArgumentError("the number of grid must be larger than 1")) 
    end

    d = Normal() # standard normal distribution
    # step 2. decide max and min value of grid    
    σ_z = sqrt(σ^2/(1.0-ρ^2))
    zmin = -s*σ_z
    zmax = s*σ_z
    z = collect(linspace(zmin,zmax,nz))
    w = z[2] -z[1] # step3. store width of between grid points
    
    # step 4. Get transition matrix
    Π = zeros(nz,nz) # transition matrix
    for row in 1:nz # Do end points first
        @inbounds Π[row, 1] = cdf(d,(z[1] - ρ*z[row] + w/2) / σ)
        @inbounds Π[row, nz] = ccdf(d,(z[nz] - ρ*z[row] - w/2) / σ) 
                               #1.0- cdf((z[nz] - ρ*z[row] - w/2) / σ)
    end

    for row in 1:nz # fill in the middle columns
        for col in 2:nz-1 
            @inbounds Π[row, col] = (cdf(d,(z[col] - ρ*z[row] + w/2) / σ) 
                                      -cdf(d,(z[col] - ρ*z[row] - w/2) / σ))
        end
    end
    return z,Π
end

mytauchen (generic function with 2 methods)

In [3]:
#########################
# step 1. Define parameters
##########################

nz = 5 # number of grid point
ρ = 0.4 # persistence 
σ = 0.4 # standard deviation

# check results

# mytaucen
z, Π = mytauchen(nz,ρ,σ)
println("Mytauchen")
println(z)
println(Π)

# Quant Econ version
println("Tauchen QEver")
x =tauchen(nz,ρ,σ)
println(x.state_values)
println(x.p)

Mytauchen
[-1.30931, -0.654654, 0.0, 0.654654, 1.30931]
[0.125971 0.562312 0.295033 0.0166006 8.3522e-5; 0.0359068 0.399091 0.494622 0.0694428 0.000936689; 0.00704518 0.199543 0.586824 0.199543 0.00704518; 0.000936689 0.0694428 0.494622 0.399091 0.0359068; 8.3522e-5 0.0166006 0.295033 0.562312 0.125971]
Tauchen QEver
[-1.30931, -0.654654, 0.0, 0.654654, 1.30931]
[0.125971 0.562312 0.295033 0.0166006 8.3522e-5; 0.0359068 0.399091 0.494622 0.0694428 0.000936689; 0.00704518 0.199543 0.586824 0.199543 0.00704518; 0.000936689 0.0694428 0.494622 0.399091 0.0359068; 8.3522e-5 0.0166006 0.295033 0.562312 0.125971]


In [4]:
###################################################
# Accuracy Test
###################################################
nz = 5 # number of grid point
ρ = 0.2 # persistence   
σ = 0.4 # standard deviation
x =tauchen(nz,ρ,σ)
X = simulate(x, 10000)
T = X[1001:end]
estρ = autocor(T,[1])
estσ = sqrt.((1.0-(estρ.^2.0)).*var(T))
println("original ρ: $ρ, original σ: $σ")
println("estimated ρ: $estρ, estimated σ: $estσ")

original ρ: 0.2, original σ: 0.4
estimated ρ: [0.208084], estimated σ: [0.43281]


### Rouwenhorst 

1. Set grid size, $N$ of Markov process, $\rho$ and $\sigma$
2. Set upper and lower bound of realization 
 and create equally spaced grid.
    
    2.1. Let $\psi = \sigma_z \sqrt{N-1}$. Set zmin $= -\psi$ and zmax$=\psi$
 
3. Compute transiton matrix, $\Pi$

4. Devide all rows except the top and bottom by 2 to make the sum of elements 1


### How to compute trantion matrix

N=2
\begin{equation*}
    \Pi_2 = \left[
    \begin{array}{ccc}
      p & 1-p  \\
      1-p & p  
    \end{array}
  \right]
\end{equation*}
For N$\ge 3$

\begin{equation*}
    \Pi_N = p \left[
    \begin{array}{ccc}
      \Pi_{N-1}  & \bf{0_N}  \\
      \bf{0}^T_N & 0  
    \end{array}
  \right]
  +(1-p) \left[
    \begin{array}{ccc}
      \Pi_{N-1}  & \bf{0_N}  \\
      \bf{0}^T_N & 0  
    \end{array}
  \right]
\end{equation*}
\begin{equation*}
    +(1-p) \left[
    \begin{array}{ccc}
      \bf{0^T_N}  & 0  \\
      \Pi_{N-1} & \bf{0_N}  
    \end{array}
  \right]
   +p \left[
    \begin{array}{ccc}
      0  & \bf{0^T_N}  \\
      \bf{0}_N & \Pi_{N-1}  
    \end{array}
  \right]
\end{equation*}

In [5]:
function myrouwenhorst(N::Int,ρ,σ)
        
    # step 2. create state grid
    const σ_z = σ/sqrt(1.0-ρ^2)
    const ψ = sqrt(N-1)* σ_z # end points
    # create shock grid
    zgrid = collect(linspace(-ψ,ψ,N)) 

    # step 3. compute trandition matrix
    const p = (1.0+ρ)/2.0
    Π   =  [p 1-p;1-p p]
     
    if N==1
        throw(ArgumentError("the number of grid must be larger than 1")) 
    elseif N==2 
        return zgrid, Π
    else 
        @inbounds for i in 3:N
            zero_v = zeros(i-1,1)
            zero_v_long = zeros(1,i)
            Π = p.*[Π zero_v; zero_v_long] +
                 (1-p).*[zero_v Π; zero_v_long] +
                 (1-p).*[zero_v_long; Π zero_v] +
                 p.*[zero_v_long; zero_v Π] 
            @views Π[2:end-1,:] ./=  2.0    # step 4. devide matrix by 2 except the top and bottom
        end
    end
    return zgrid, Π
end

myrouwenhorst (generic function with 1 method)

In [6]:
# step 1. Define parameters
nz = 5# number of grid point
ρ = 0.2 # persistence 
σ = 0.4 # standard deviation

# check results

# my code
zgrid,Π = myrouwenhorst(nz,ρ,σ)
println("Result of my code")
println(zgrid)
println(Π)

# QE ver
println("Result of QE")
mc =rouwenhorst(nz,ρ,σ) 
println(mc.state_values)
println(mc.p)

Result of my code
[-0.816497, -0.408248, 0.0, 0.408248, 0.816497]
[0.1296 0.3456 0.3456 0.1536 0.0256; 0.0864 0.3024 0.3744 0.1984 0.0384; 0.0576 0.2496 0.3856 0.2496 0.0576; 0.0384 0.1984 0.3744 0.3024 0.0864; 0.0256 0.1536 0.3456 0.3456 0.1296]
Result of QE
-0.8164965809277261:0.4082482904638631:0.8164965809277261
[0.1296 0.3456 0.3456 0.1536 0.0256; 0.0864 0.3024 0.3744 0.1984 0.0384; 0.0576 0.2496 0.3856 0.2496 0.0576; 0.0384 0.1984 0.3744 0.3024 0.0864; 0.0256 0.1536 0.3456 0.3456 0.1296]


In [7]:
###################################################
# Accuracy Test
###################################################
nz = 5 # number of grid point
ρ = 0.2 # persistence 
σ = 0.4 # standard deviation
x =rouwenhorst(nz,ρ,σ)
X = simulate(x, 10000)
T = X[1001:end]
estρ = autocor(T,[1])
estσ = sqrt.((1.0-(estρ.^2.0)).*var(T))
println("original ρ: $ρ, original σ: $σ")
println("estimated ρ: $estρ, estimated σ: $estσ")

original ρ: 0.2, original σ: 0.4
estimated ρ: [0.215598], estimated σ: [0.400428]


### Stationary distribution
The unconditional probability distribution $x_t$ evolves
$$x_{t+1}=\Pi' \cdot x_t$$

The probability distribution is stationary if it satisfies  $x_{t+1}=x_t \,\,\, \text{for all $t$} $

Is that distribution unique? $\rightarrow$ yes.
#### Theorem

Let P be a stochastic matrix with $P_{ij} >0,\, \forall(i,j)$. Then P has a unique stationary distribution and, the process is asymptotically stationary.


### How to compute the invariant distribution
By previous theorem, any initial distributions attain to the same stationary distribution.

1. guess initial distribution
2. update distribution using transition matrix
3. if it converges, stop updating, else go to step 2.

In [8]:
function inv_dist(zgrid,Π)
    const nz = size(zgrid,1)
    stdist =    ones(nz)./nz
    stdist1 = zeros(nz)
    err = 1.0
    const tol = 1e-8
    Trans = transpose(Π)

    while err > tol
        stdist1 = Trans*stdist
        err=maximum(abs.(stdist1-stdist))
        stdist = copy(stdist1)  
    end

    return stdist
end

inv_dist (generic function with 1 method)

In [9]:
# check invariant distribution

# my code
nz = 5 # number of grid point
ρ = 0.2 # persistence 
σ = 0.4 # standard deviation
x,y = myrouwenhorst(nz,ρ,σ)
stat_dist=inv_dist(x,y)
println("$stat_dist")

# QEver
mc = rouwenhorst(nz,ρ,σ)
st = stationary_distributions(mc)

[0.0625, 0.25, 0.375, 0.25, 0.0625]


1-element Array{Array{Float64,1},1}:
 [0.0625, 0.25, 0.375, 0.25, 0.0625]